<a href="https://colab.research.google.com/github/DavideScassola/PML2024/blob/main/./Notebooks/09_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook 9: Variational Inference Exercise

Using the following dataset about weather in Sidney to predict if tomorrow it will rain:
1. Fit a bayesian logistic regression model with VI (using as predictors only the numerical columns)
2. Compare the Bayesian prediction with the frequentist prediction (for the frequentist you can use the mean of the guide)

Hint: you may need to call `.to_event()` when defining a distribution in the model, see [this page](https://pyro.ai/examples/tensor_shapes.html) for more information.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch

# Loading the data
sydney_data = pd.read_csv('https://raw.githubusercontent.com/r-doz/PML2025/main/data/weatherSydney.csv')
sydney_data

HTTPError: HTTP Error 404: Not Found

In [2]:
# extract predictors and target variable
numerical_columns = sydney_data.select_dtypes(include=[np.number])
y = sydney_data['RainTomorrow']

X = torch.from_numpy(numerical_columns.to_numpy()).float()
y = torch.from_numpy((y== "Yes").to_numpy()).float()

# Split train and test
def split_and_standardize(*, X, y, ratio=0.05):
    
    # Splitting train and test sets
    cut = int(ratio * len(y))
    X_train = X[:cut]
    y_train = y[:cut]
    
    X_test = X[cut:]
    y_test = y[cut:]
    
    # Standardizing data
    x_mean = X_train.mean(axis=0)
    x_std = X_train.std(axis=0)
    
    X_train = (X_train - x_mean) / x_std
    X_test = (X_test - x_mean) / x_std
    
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = split_and_standardize(X=X, y=y)

# Add bias term
X_train = torch.hstack([torch.ones((X_train.shape[0], 1)), X_train])
X_test = torch.hstack([torch.ones((X_test.shape[0], 1)), X_test])

In [3]:
X_train

tensor([[ 1.0000,  0.0237, -0.2067,  ..., -1.2894,  0.2598,  0.0224],
        [ 1.0000,  0.1077,  0.1015,  ...,  0.2579,  0.3972,  0.4307],
        [ 1.0000,  0.5273,  0.4867,  ..., -0.1289,  0.6719,  0.6758],
        ...,
        [ 1.0000, -0.6477, -0.2580,  ..., -0.9026, -0.4271, -0.7671],
        [ 1.0000, -1.1093, -0.8487,  ..., -1.2894, -1.0747, -1.0122],
        [ 1.0000, -0.8575,  0.2813,  ..., -1.2894, -0.8981, -0.1954]])

In [4]:
y_train

tensor([0., 0., 0., 0., 1., 0., 1., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])